# FM 24 Moneyball

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.express as px
import re

# Import created functions
import sys
sys.path.append(r'C:\Users\kevmi\Documents\FM 24 Moneyball\FM-24-Moneyball')

from money_ball_util import *

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Show all columns in pandas
pd.set_option('display.max_columns', 500) 

%matplotlib inline

## Ingest and clean data

In [83]:
### Read raw data

# Select file path for raw data
path = r"C:\Users\kevmi\Documents\FM 24 Moneyball\FM Data Lab\Man Utd\FM 24 exports\Summer 2024\Defensive Midfielders.html"

# Ingest in a dataframe
df_players_raw = pd.read_html(path, encoding='utf-8')[0]

# Format column names 
df_players_raw.columns = [x.lower().strip().replace(' ', '_').replace('-', '_') for x in df_players_raw.columns]

# ---------------------------------------------------------------------------------------------------------------------

### Read processed Data lab data

# Select file path for processed data
path = r"C:\Users\kevmi\Documents\FM 24 Moneyball\FM Data Lab\Man Utd\FM Data Lab outputs\Summer 2024\Defensive Midfielders.csv"

# Ingest in a dataframe 
df_players = pd.read_csv(path)   

# Format column names 
df_players.columns = [x.lower().strip().replace(' ', '_').replace('-', '_') for x in df_players.columns]

# Bring ID from raw data
df_players = df_players.merge(df_players_raw[['name', 'age', 'height', 'weight', 'uid']], 
                 left_on=['name', 'age', 'height', 'weight'], 
                 right_on=['name', 'age', 'height', 'weight']).reset_index().rename(columns={'level_0': 'id'})

df_players['id'] = df_players['uid']

df_players.drop(columns='uid', inplace=True)

## Data cleaning

In [84]:
df_players.head()

,id,name,position,age,height,weight,inf,club,division,nationality,home_grown,personality,media_handling,wage,transfer_value,asking_price,preferred_foot,starts,minutes_played,average_rating,sub_appearances,minutes/game,index,entries,leaguemultiplier,passes_attempted/90,passes_completed/90,pass_completion_%,progressive_passes/90,progressive_pass_rate,key_passes/90,key_pass_%,open_play_key_passes/90,open_play_key_pass_%,chances_created/90,clear_cut_chances_created/90,clear_cut_chances_%,pressures_attempted/90,pressures_completed/90,pressure_success_%,possession_won/90,possession_lost/90,poss+_/90,poss+__%,tackles_attempted/90,tackles_completed/90,tackle_completion_%,tackles_failed/90,key_tackles/90,key_tackles_%,tackle_quality,interceptions/90,blocks/90,shots_blocked/90,headers_attempted/90,headers_won/90,headers_won_%,headers_lost/90,headers_lost_%,key_headers/90,key_headers_%,aerial_challenges_attempted/90,duels_win_%,fouls_committed/90,clearances/90,distance_covered/90,mistakes_leading_to_goal,sprints/90,attacking_actions/90,creative_actions/90,defensive_actions/90,excitement_factor/90,general_performance,defensive_defender,creative_defender,attacking_defender,creative_midfielder,attacking_midfielder,creative_winger,attacking_winger,creative_forward,attacking_forward,finisher,aerial_threat,reader,assister
0,27025242,Nicolai Poulsen,"DM, M (C)",30,176 cm,70 kg,-,AGF,3F Superliga,DEN,-,-,Scouting Required,"£4,600 p/w",£300K - £2.9M,-,Right Only,26,2318,6.87,1,85.85,0,1,False,65.58,58.20,89,6.56,10.01,1.24,1.89,1.20,1.84,0.81,0.31,38.10,11.96,3.34,27.92,8.70,9.75,-1.05,-10.77,4.04,3.11,77,0.93,0.08,2.50,2.04,1.40,0.43,0.19,3.49,1.40,40.00,2.10,60.00,0.08,5.56,3.49,59.79,1.98,1.32,13.52,0,14.64,0.99,1.67,11.87,2.87,34,66,56,33,35,21,26,29,30,29,16,35,29,44
1,2000100721,Julius Beck,"DM, M (C)",19,178 cm,62 kg,-,AGF,3F Superliga,DEN,-,Balanced,Media-friendly,"£4,700 p/w",£4M - £8M,-,Right,16,1424,6.78,3,74.95,1,1,False,56.57,50.44,89,5.12,9.05,0.88,1.56,0.88,1.56,0.20,0.19,100.00,11.00,3.48,31.61,8.53,7.46,1.07,14.34,2.91,2.09,72,0.82,0.00,0.00,2.30,2.02,0.44,0.25,2.78,1.14,40.91,1.64,59.09,0.00,0.00,2.78,56.67,1.26,1.71,13.22,0,10.05,0.67,1.20,10.45,2.59,2,44,21,5,7,4,5,5,7,6,3,19,39,11
2,92082660,Michael Akoto,"D (RC), WB (R), DM, M (C)",26,188 cm,72 kg,-,AGF,3F Superliga,GER (GHA),-,-,Scouting Required,"£4,000 p/w",£230K - £2.3M,-,Right,20,1872,6.82,8,66.86,2,1,False,59.47,53.61,90,5.24,8.81,0.58,0.97,0.58,0.97,0.06,0.05,100.00,6.59,2.07,31.39,11.49,6.44,5.05,78.42,2.16,1.68,78,0.48,0.38,22.86,2.65,2.60,0.48,0.24,6.11,3.75,61.42,2.36,38.58,0.14,3.85,6.11,65.70,0.82,0.87,12.26,0,6.88,0.97,0.77,15.45,2.69,18,96,24,19,31,43,22,20,17,32,71,68,96,3
3,53159880,Sivert Mannsverk,"DM, M (C)",22,185 cm,76 kg,Inj,Ajax,Eredivisie,NOR,-,-,Scouting Required,"£12,750 p/w",£12.5M - £20M,-,Right,36,3004,6.82,1,81.19,3,1,False,55.07,47.85,87,5.30,9.63,1.17,2.12,0.96,1.74,1.17,0.36,30.77,10.88,2.97,27.27,9.11,9.68,-0.57,-5.89,3.09,2.04,66,1.05,0.00,0.00,2.24,1.74,0.39,0.21,4.07,2.52,61.76,1.56,38.24,0.39,15.48,4.07,63.60,1.38,0.78,13.01,0,7.49,1.25,1.50,12.86,3.69,18,15,32,19,25,11,37,29,34,23,22,75,52,32
4,37073317,Kenneth Taylor,"DM, M/AM (C)",22,182 cm,66 kg,-,Ajax,Eredivisie,NED,-,Fairly Determined,Level-headed,"£21,500 p/w",£75M - £89M,-,Left,42,3579,7.01,0,85.21,4,1,False,46.47,40.31,87,4.02,8.66,1.28,2.76,0.88,1.89,0.91,0.38,41.67,11.42,3.02,26.43,7.97,8.98,-1.01,-11.25,3.22,2.62,81,0.60,0.03,0.96,5.12,1.56,0.60,0.20,4.85,2.11,43.52,2.74,56.48,0.15,7.14,4.85,58.57,0.63,0.50,13.24,0,10.96,2.44,1.48,11.04,4.95,84,45,35,76,76,81,79,83,83,84,98,56,25,60


In [85]:
# # Keep only natural DMs
# path = 'data/Natural DMs.html'

# # Ingest in a dataframe
# df_dms = pd.read_html(path, encoding='utf-8')[0]

# # Format column names 
# df_dms.columns = [x.lower().strip().replace(' ', '_').replace('-', '_') for x in df_dms.columns]

# df_dms['natural_dm'] = 'Yes'

# df_dms = df_dms[['uid', 'natural_dm']]

# df_players = df_players.merge(df_dms, 
#                               left_on='id',
#                               right_on='uid',
#                               how='left')

# df_players = df_players[df_players['natural_dm'] == 'Yes']

In [ ]:
# Apply to your DataFrame
df_players['price_estimate_M'] = df_players['transfer_value'].apply(parse_transfer_value) / 1000000

## Passing analysis

### Full universe

In [88]:
# Initialize list of targets
targets = df_players['id']

scatter_plot(df_players,
             x_metric='passes_attempted/90', 
             y_metric='progressive_pass_rate',
             z_metric='pass_completion_%',
             title='Passing quality', 
             show_name_label=False
)

In [89]:
# Filter data 
mask = (
        (df_players['passes_attempted/90'] >= 45) & \
        (df_players['progressive_pass_rate'] >= 7) & \
        (df_players['pass_completion_%'] >= 85) & \
        (df_players['age'] <= 26) & \
        (df_players['id'].isin(targets))
        
        
)

df_targets = df_players[mask]

scatter_plot(df_targets,
             x_metric='passes_attempted/90', 
             y_metric='progressive_pass_rate',
             z_metric='pass_completion_%',
             title='Passing quality', 
             show_name_label=False
)

# Keep only players that fulfill criteria
targets = list(df_targets['id'])

## Possession Analysis

In [90]:
# Full universe 
scatter_plot(df_targets,
             x_metric='possession_lost/90', 
             y_metric='possession_won/90',
             z_metric='poss+_/90',
             title='Possesion quality', 
             show_name_label=False
)

In [91]:
# Filter data 
mask = (
        (df_players['possession_lost/90'] <= 8) & \
        (df_players['possession_won/90'] >= 7.5) & \
        (df_players['poss+_/90'] >= 0) & \
        (df_players['age'] <= 26) & \
        (df_players['id'].isin(targets))
        
        
)

df_targets = df_players[mask]

scatter_plot(df_targets,
             x_metric='possession_lost/90', 
             y_metric='possession_won/90',
             z_metric='poss+_/90',
             title='Possesion quality', 
             show_name_label=False
)

# Keep only players that fulfill criteria
targets = list(df_targets['id'])

## Aggresion vs Control
Identify aggresive and foul prones targets

In [92]:
# Full universe 
scatter_plot(df_targets,
             x_metric='tackles_completed/90', 
             y_metric='fouls_committed/90',
             z_metric='tackle_completion_%',
             title='Aggresion - Control', 
             show_name_label=False
)

In [93]:
# Compute tackes/fouls ratio
df_players['tackles_comp-fouls/90'] = df_players['tackles_completed/90'] / df_players['fouls_committed/90']
# Filter data 
mask = (
        (df_players['tackles_comp-fouls/90'] <= 2) & \
        (df_players['tackles_completed/90'] >= 1.5) & \
        (df_players['tackle_completion_%'] >= 65) & \
        #(df_players['price_estimate_M'] <= 40) & \
        (df_players['id'].isin(targets))
        
        
)

df_targets = df_players[mask]

scatter_plot(df_targets,
             x_metric='tackles_completed/90', 
             y_metric='fouls_committed/90',
             z_metric='tackle_completion_%',
             title='Aggresion - Control', 
             show_name_label=False
)

# Keep only players that fulfill criteria
targets = df_targets['id']

## Pressure quality

In [94]:
# Full universe 
scatter_plot(df_targets,
             x_metric='pressures_completed/90', 
             y_metric='pressure_success_%',
             z_metric='duels_win_%',
             title='Pressure quality', 
             show_name_label=True
)

In [95]:
# Filter data 
mask = (
        (df_players['pressures_completed/90'] >= 2.0) & \
        (df_players['pressure_success_%'] >= 20) & \
        (df_players['duels_win_%'] >= 50) & \
        (df_players['id'].isin(targets)) | \
        (df_players['name'].str.contains('Barreiro')) 
        
        
)

df_targets = df_players[mask]

scatter_plot(df_targets,
             x_metric='pressures_completed/90', 
             y_metric='pressure_success_%',
             z_metric='duels_win_%',
             title='Pressure quality', 
             show_name_label=True
)

# Keep only players that fulfill criteria
targets = df_targets['id']

## Presence

In [96]:
# Full universe 
scatter_plot(df_targets,
             x_metric='interceptions/90', 
             y_metric='blocks/90',
             z_metric='clearances/90',
             title='Presence', 
             show_name_label=True
) 

In [97]:
df_players[df_players['name'] == 'Casemiro']

,id,name,position,age,height,weight,inf,club,division,nationality,home_grown,personality,media_handling,wage,transfer_value,asking_price,preferred_foot,starts,minutes_played,average_rating,sub_appearances,minutes/game,index,entries,leaguemultiplier,passes_attempted/90,passes_completed/90,pass_completion_%,progressive_passes/90,progressive_pass_rate,key_passes/90,key_pass_%,open_play_key_passes/90,open_play_key_pass_%,chances_created/90,clear_cut_chances_created/90,clear_cut_chances_%,pressures_attempted/90,pressures_completed/90,pressure_success_%,possession_won/90,possession_lost/90,poss+_/90,poss+__%,tackles_attempted/90,tackles_completed/90,tackle_completion_%,tackles_failed/90,key_tackles/90,key_tackles_%,tackle_quality,interceptions/90,blocks/90,shots_blocked/90,headers_attempted/90,headers_won/90,headers_won_%,headers_lost/90,headers_lost_%,key_headers/90,key_headers_%,aerial_challenges_attempted/90,duels_win_%,fouls_committed/90,clearances/90,distance_covered/90,mistakes_leading_to_goal,sprints/90,attacking_actions/90,creative_actions/90,defensive_actions/90,excitement_factor/90,general_performance,defensive_defender,creative_defender,attacking_defender,creative_midfielder,attacking_midfielder,creative_winger,attacking_winger,creative_forward,attacking_forward,finisher,aerial_threat,reader,assister,price_estimate_M,tackles_comp-fouls/90
287,19061518,Casemiro,"DM, M (C)",32,185 cm,84 kg,-,Man Utd,Premier League,BRA (ESP),-,Spirited,Evasive,"£300,000 p/w",£41M - £51M,-,Right,35,2888,6.78,11,62.78,287,1,False,59.15,53.38,90,4.49,7.59,0.84,1.42,0.84,1.42,0.31,0.16,50.0,8.94,2.4,26.83,8.48,6.95,1.53,22.01,3.49,2.59,74,0.9,0.12,4.82,1.81,1.46,0.37,0.22,3.8,2.59,68.03,1.22,31.97,0.62,24.1,3.8,70.94,1.93,0.87,13.55,1,13.65,0.78,1.06,15.84,2.31,10,67,13,5,7,3,7,5,8,4,0,83,72,18,55.2,1.341969


In [98]:
# Filter data 
mask = (
        (df_players['interceptions/90'] >= 1.4) & \
        (df_players['blocks/90'] >= 0.25) & \
        (df_players['clearances/90'] >= 0.5) & \
        (df_players['id'].isin(targets))
        
        
)

df_targets = df_players[mask]

scatter_plot(df_targets,
             x_metric='interceptions/90', 
             y_metric='blocks/90',
             z_metric='clearances/90',
             title='Presence', 
             show_name_label=True
)

#df_targets = df_players[mask]

## Distance

In [99]:
# Full universe 
scatter_plot(df_targets,
             x_metric='distance_covered/90', 
             y_metric='defensive_actions/90',
             z_metric='creative_actions/90',
             title='Distance', 
             show_name_label=True
) 

In [110]:
df_targets.groupby('division')['id'].count()

division
3F Superliga                           2
Bundesliga                             6
Campeonato Brasileiro Série A Assaí    1
Eredivisie                             3
LaLiga EA Sports                       3
Liga Portugal Betclic                  6
Liga Profesional de Fútbol             5
Ligue 1 Uber Eats                      1
Premier League                         3
Serie A TIM                            3
Sky Bet Championship                   4
Name: id, dtype: int64

In [112]:
df_targets.groupby('division').get_group('Sky Bet Championship').sort_values('age')

,id,name,position,age,height,weight,inf,club,division,nationality,home_grown,personality,media_handling,wage,transfer_value,asking_price,preferred_foot,starts,minutes_played,average_rating,sub_appearances,minutes/game,index,entries,leaguemultiplier,passes_attempted/90,passes_completed/90,pass_completion_%,progressive_passes/90,progressive_pass_rate,key_passes/90,key_pass_%,open_play_key_passes/90,open_play_key_pass_%,chances_created/90,clear_cut_chances_created/90,clear_cut_chances_%,pressures_attempted/90,pressures_completed/90,pressure_success_%,possession_won/90,possession_lost/90,poss+_/90,poss+__%,tackles_attempted/90,tackles_completed/90,tackle_completion_%,tackles_failed/90,key_tackles/90,key_tackles_%,tackle_quality,interceptions/90,blocks/90,shots_blocked/90,headers_attempted/90,headers_won/90,headers_won_%,headers_lost/90,headers_lost_%,key_headers/90,key_headers_%,aerial_challenges_attempted/90,duels_win_%,fouls_committed/90,clearances/90,distance_covered/90,mistakes_leading_to_goal,sprints/90,attacking_actions/90,creative_actions/90,defensive_actions/90,excitement_factor/90,general_performance,defensive_defender,creative_defender,attacking_defender,creative_midfielder,attacking_midfielder,creative_winger,attacking_winger,creative_forward,attacking_forward,finisher,aerial_threat,reader,assister,price_estimate_M,tackles_comp-fouls/90
118,28108493,Joel Latibeaudiere,"D (RC), DM",24,181 cm,80 kg,Wnt,Coventry,Sky Bet Championship,JAM (ENG),Trained in nation (15-21),-,Scouting Required,"£8,000 p/w",£750K,£725K,Right,15,1600,6.64,15,53.33,118,1,False,60.52,54.67,90,7.26,11.99,0.62,1.02,0.62,1.02,0.00,0.00,0.00,8.04,2.36,29.37,9.96,7.82,2.14,27.37,2.64,2.31,87,0.34,0.11,4.88,1.15,1.52,0.62,0.39,7.09,4.11,57.94,2.98,42.06,0.84,20.55,7.09,65.90,2.31,1.69,12.73,1,10.35,0.69,0.73,19.43,2.48,23,97,12,17,22,27,22,19,22,21,35,99,87,3,0.750,1.000000
227,29170352,David Kasumu,"D/WB (R), DM, M (C)",24,174 cm,60 kg,-,Huddersfield,Sky Bet Championship,NGA (ENG),Trained in nation (15-21),-,Scouting Required,"£6,000 p/w",£550K - £5.4M,-,Right,35,2843,6.79,0,81.23,227,1,False,47.71,42.29,89,4.69,9.82,0.73,1.53,0.73,1.53,0.23,0.22,100.00,11.02,2.60,23.56,9.18,6.90,2.28,33.04,3.55,2.66,75,0.89,0.06,2.38,1.93,1.68,0.57,0.38,3.23,0.89,27.45,2.34,72.55,0.16,17.86,3.23,52.34,1.84,1.65,13.43,3,10.42,1.03,1.01,11.57,2.41,35,65,9,17,22,18,20,21,41,19,50,36,45,32,3.570,1.445652
437,29156436,Flynn Downes,"DM, M (C)",25,182 cm,79 kg,-,Southampton,Sky Bet Championship,ENG,Trained in nation (15-21),Perfectionist,Media-friendly,"£30,000 p/w",£8M - £11.5M,-,Right Only,43,3690,6.90,1,83.86,437,1,False,62.41,56.98,91,7.51,12.04,0.85,1.37,0.85,1.37,0.27,0.24,90.91,8.66,2.49,28.73,8.12,6.46,1.66,25.70,3.05,2.34,77,0.71,0.02,1.04,1.39,1.49,0.32,0.20,2.68,1.46,54.55,1.22,45.45,0.17,11.67,2.68,66.38,2.20,0.51,13.20,1,9.85,1.37,1.29,9.80,2.56,82,25,63,60,48,65,48,61,37,53,11,42,22,84,11.700,1.063636
117,67228750,Keidi Bare,"DM, M (C)",26,174 cm,66 kg,Int,Coventry,Sky Bet Championship,ALB,-,-,Scouting Required,"£6,500 p/w",£35K - £375K,-,Right,37,2627,6.85,0,71.00,117,1,False,56.49,51.15,91,6.00,10.61,0.96,1.70,0.89,1.58,0.21,0.21,100.00,11.03,3.08,27.95,8.63,7.43,1.20,16.15,3.63,2.84,78,0.79,0.07,2.41,2.04,2.06,0.41,0.21,3.49,0.99,28.43,2.50,71.57,0.14,13.79,3.49,53.85,1.78,1.23,13.43,0,10.59,1.59,1.30,12.00,3.84,55,85,74,42,46,40,59,39,36,32,29,37,59,63,0.246,1.595506
